In [3]:
from bioservices import UniProt
import pandas as pd
from Bio import SeqIO
import urllib
import re
import ast

### Get UniProt entry id and feature info for EC numbers for top hundred organisms

In [4]:
#Get cleaned data from BRENDA
df_brenda = pd.read_csv('BRENDA_interactions_intracellular.txt', index_col=0, header=0)

In [5]:
df_brenda

,EC,Enz,Met,Org,Mode,ChEBI
0,1.1.1.1,alcohol dehydrogenase,2-propanol,Sulfolobus acidocaldarius,+,CHEBI:17824
1,1.1.1.1,alcohol dehydrogenase,ethanol,Saccharomyces cerevisiae,+,CHEBI:16236
2,1.1.1.1,alcohol dehydrogenase,Isopropanol,Saccharomyces cerevisiae,+,CHEBI:17824
3,1.1.1.1,alcohol dehydrogenase,Urea,Thermus sp.,+,CHEBI:16199
4,1.1.1.101,acylglycerone-phosphate reductase,dihydroxyacetone,Saccharomyces cerevisiae,+,CHEBI:16016
...,...,...,...,...,...,...
33238,7.6.2.9,ABC-type quaternary amine transporter,Betaine aldehyde,Aphanothece halophytica,-,CHEBI:15710
33239,7.6.2.9,ABC-type quaternary amine transporter,carnitine,Lactococcus lactis,-,CHEBI:3424
33240,7.6.2.9,ABC-type quaternary amine transporter,choline,Aphanothece halophytica,-,CHEBI:15354
33241,7.6.2.9,ABC-type quaternary amine transporter,proline,Lactococcus lactis,-,CHEBI:60039


In [6]:
#Get the top hundred organisms from previously created file
top_hundred_list = pd.read_csv('top_hundred_organisms.txt', sep='\t', index_col=0)['Org'].tolist()

In [7]:
#Filter dataframe for interactions in the top hundred organisms
df_brenda_top_hundred = df_brenda[df_brenda['Org'].isin(top_hundred_list)]

In [12]:
df_brenda_top_hundred

,EC,Enz,Met,Org,Mode,ChEBI,Entry_string
0,1.1.1.1,alcohol dehydrogenase,2-propanol,Sulfolobus acidocaldarius,+,CHEBI:17824,EC:1.1.1.1+AND+Sulfolobus acidocaldarius+AND+r...
1,1.1.1.1,alcohol dehydrogenase,ethanol,Saccharomyces cerevisiae,+,CHEBI:16236,EC:1.1.1.1+AND+Saccharomyces cerevisiae+AND+re...
2,1.1.1.1,alcohol dehydrogenase,Isopropanol,Saccharomyces cerevisiae,+,CHEBI:17824,EC:1.1.1.1+AND+Saccharomyces cerevisiae+AND+re...
4,1.1.1.101,acylglycerone-phosphate reductase,dihydroxyacetone,Saccharomyces cerevisiae,+,CHEBI:16016,EC:1.1.1.101+AND+Saccharomyces cerevisiae+AND+...
5,1.1.1.101,acylglycerone-phosphate reductase,ethanol,Saccharomyces cerevisiae,+,CHEBI:16236,EC:1.1.1.101+AND+Saccharomyces cerevisiae+AND+...
...,...,...,...,...,...,...,...
33234,7.6.2.3,ABC-type glutathione-S-conjugate transporter,leukotriene C4,Homo sapiens,-,CHEBI:16978,EC:7.6.2.3+AND+Homo sapiens+AND+reviewed:true
33237,7.6.2.9,ABC-type quaternary amine transporter,ADP,Lactococcus lactis,-,CHEBI:16761,EC:7.6.2.9+AND+Lactococcus lactis+AND+reviewed...
33239,7.6.2.9,ABC-type quaternary amine transporter,carnitine,Lactococcus lactis,-,CHEBI:3424,EC:7.6.2.9+AND+Lactococcus lactis+AND+reviewed...
33241,7.6.2.9,ABC-type quaternary amine transporter,proline,Lactococcus lactis,-,CHEBI:60039,EC:7.6.2.9+AND+Lactococcus lactis+AND+reviewed...


#### Add column of entry-string for search in UniProt

In [9]:
#Define formula for making entry-string
def entry_formula(EC, Org):
    return f'EC:{EC}+AND+{Org}+AND+reviewed:true'

In [11]:
#apply formula to dataframe
df_brenda_top_hundred['Entry_string'] = df_brenda_top_hundred.apply(lambda x: entry_formula(x['EC'], x['Org']), axis=1)

C:\Users\Elin\AppData\Local\Temp\ipykernel_2644\2498833545.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_brenda_top_hundred['Entry_string'] = df_brenda_top_hundred.apply(lambda x: entry_formula(x['EC'], x['Org']), axis=1)


In [13]:
#Make dataframe of only enzymes (EC numbers), organisms and entry-strings
df_ec_enz_org = df_brenda_top_hundred[['EC', 'Enz', 'Org', 'Entry_string']].drop_duplicates()

##### Search in UniProt

In [15]:
u = UniProt(verbose=False)

In [16]:
#Get info for every EC number+Organism combination
#The get_df() function has a limit of 10 entries per search for the sake of efficiency, 
# since the number of entries is quite large I decided to not change that.
#Also: the UniProt database changes all the time, and a new search would therefore probably not give the same results

entries = set(df_ec_enz_org['Entry_string'])

df = pd.DataFrame([])

for entry in entries:
    df = pd.concat([df,u.get_df(entry)])

In [16]:
feature_dfs = []

entries = set(df_ec_enz_org['Entry_string'])

for entry in entries:
    feature_dfs.append(u.get_df(entry, limit=100))

feature_dfs

[Empty DataFrame
 Columns: [Entry, Entry Name, Gene Names, Gene Names (primary), Gene Names (synonym), Gene Names (ordered locus), Gene Names (ORF), Organism, Organism (ID), Protein names, Proteomes, Taxonomic lineage, Virus hosts, Fragment, Sequence, Length, Mass, Gene encoded by, Alternative products (isoforms), Erroneous gene model prediction, Mass spectrometry, Polymorphism, RNA Editing, Sequence caution, Alternative sequence, Natural variant, Non-adjacent residues, Non-standard residue, Non-terminal residue, Sequence conflict, Sequence uncertainty, Sequence version, Coiled coil, Compositional bias, Domain [CC], Domain [FT], Motif, Protein families, Region, Repeat, Zinc finger, Absorption, Active site, Activity regulation, Binding site, Catalytic activity, Cofactor, DNA binding, EC number, Function [CC], Kinetics, Pathway, pH dependence, Redox potential, Site, Temperature dependence, Gene Ontology (GO), Gene Ontology (biological process), Gene Ontology (molecular function), Gene On

In [17]:
df = pd.concat(feature_dfs, axis=0)

df

,Entry,Entry Name,Gene Names,Gene Names (primary),Gene Names (synonym),Gene Names (ordered locus),Gene Names (ORF),Organism,Organism (ID),Protein names,...,Glycosylation,Initiator methionine,Lipidation,Modified residue,Peptide,Post-translational modification,Propeptide,Signal peptide,Transit peptide,PDB
0,Q7YRC6,AURKB_BOVIN,AURKB AIK2 AIM1 AIRK2 ARK2 STK1 STK12 STK5,AURKB,AIK2 AIM1 AIRK2 ARK2 STK1 STK12 STK5,NaN,NaN,Bos taurus (Bovine),9913,Aurora kinase B (EC 2.7.11.1) (Aurora 1) (Auro...,...,NaN,NaN,NaN,"MOD_RES 35; /note=""Phosphothreonine""; /evidenc...",NaN,PTM: The phosphorylation of Thr-232 requires t...,NaN,NaN,NaN,NaN
1,Q32PI1,VRK1_BOVIN,VRK1,VRK1,NaN,NaN,NaN,Bos taurus (Bovine),9913,Serine/threonine-protein kinase VRK1 (EC 2.7.1...,...,NaN,NaN,NaN,"MOD_RES 376; /note=""Phosphoserine""; /evidence=...",NaN,PTM: Autophosphorylated at various serine and ...,NaN,NaN,NaN,NaN
2,F1MH24,AAK1_BOVIN,AAK1,AAK1,NaN,NaN,NaN,Bos taurus (Bovine),9913,AP2-associated protein kinase 1 (EC 2.7.11.1) ...,...,NaN,NaN,NaN,"MOD_RES 1; /note=""N-acetylmethionine""; /eviden...",NaN,PTM: Autophosphorylated. {ECO:0000250|UniProtK...,NaN,NaN,NaN,NaN
3,P35508,KC1D_BOVIN,CSNK1D HCKID,CSNK1D,HCKID,NaN,NaN,Bos taurus (Bovine),9913,Casein kinase I isoform delta (CKI-delta) (CKI...,...,NaN,NaN,NaN,"MOD_RES 328; /note=""Phosphoserine""; /evidence=...",NaN,PTM: Autophosphorylated on serine and threonin...,NaN,NaN,NaN,NaN
4,Q01314,AKT1_BOVIN,AKT1 PKB,AKT1,PKB,NaN,NaN,Bos taurus (Bovine),9913,RAC-alpha serine/threonine-protein kinase (EC ...,...,"CARBOHYD 129; /note=""O-linked (GlcNAc) serine;...",NaN,NaN,"MOD_RES 14; /note=""N6-acetyllysine""; /evidence...",NaN,PTM: O-GlcNAcylation at Thr-305 and Thr-312 in...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,P00731,CBPA1_RAT,Cpa1 Cpa,Cpa1,Cpa,NaN,NaN,Rattus norvegicus (Rat),10116,Carboxypeptidase A1 (EC 3.4.17.1),...,NaN,NaN,NaN,NaN,NaN,NaN,"PROPEP 17..110; /note=""Activation peptide""; /i...",SIGNAL 1..16,NaN,NaN
1,P21961,CBPA3_RAT,Cpa3,Cpa3,NaN,NaN,NaN,Rattus norvegicus (Rat),10116,Mast cell carboxypeptidase A (EC 3.4.17.1) (Ca...,...,NaN,NaN,NaN,NaN,NaN,NaN,"PROPEP 11..104; /note=""Activation peptide""; /i...","SIGNAL <1..10; /evidence=""ECO:0000250""",NaN,NaN
0,P39669,GLGC_RHIRD,glgC,glgC,NaN,NaN,NaN,Rhizobium radiobacter (Agrobacterium tumefacie...,358,Glucose-1-phosphate adenylyltransferase (EC 2....,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3BRK;5W5R;5W5T;5W6J;6VR0;
1,Q8U8L5,GLGC_AGRFC,glgC Atu4076 AGR_L_1560,glgC,NaN,Atu4076,AGR_L_1560,Agrobacterium fabrum (strain C58 / ATCC 33970)...,176299,Glucose-1-phosphate adenylyltransferase (EC 2....,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6V96;6V99;6V9A;


In [18]:
len(set(df['Entry']))

16088

In [21]:
#Reduce dataframe to fewer columns
columns_of_interest = ['Entry', 'Gene Names (primary)', 'Organism', 'Protein names', 'Domain [CC]', 'Domain [FT]', 'Motif', 'Protein families', 'Region', 'Active site', 'Activity regulation', 'Binding site']
df_features = df[columns_of_interest]

In [22]:
df_features = df_features.drop_duplicates()

In [23]:
df_features

,Entry,Gene Names (primary),Organism,Protein names,Domain [CC],Domain [FT],Motif,Protein families,Region,Active site,Activity regulation,Binding site
0,Q7YRC6,AURKB,Bos taurus (Bovine),Aurora kinase B (EC 2.7.11.1) (Aurora 1) (Auro...,NaN,"DOMAIN 77..327; /note=""Protein kinase""; /evide...",NaN,"Protein kinase superfamily, Ser/Thr protein ki...","REGION 46..65; /note=""Disordered""; /evidence=""...","ACT_SITE 200; /note=""Proton acceptor""; /eviden...",ACTIVITY REGULATION: Activity is greatly incre...,"BINDING 83..91; /ligand=""ATP""; /ligand_id=""ChE..."
1,Q32PI1,VRK1,Bos taurus (Bovine),Serine/threonine-protein kinase VRK1 (EC 2.7.1...,NaN,"DOMAIN 37..317; /note=""Protein kinase""; /evide...",NaN,"Protein kinase superfamily, CK1 Ser/Thr protei...","REGION 352..396; /note=""Disordered""; /evidence...","ACT_SITE 177; /note=""Proton acceptor""; /eviden...",ACTIVITY REGULATION: Active in presence of Mn(...,"BINDING 43..51; /ligand=""ATP""; /ligand_id=""ChE..."
2,F1MH24,AAK1,Bos taurus (Bovine),AP2-associated protein kinase 1 (EC 2.7.11.1) ...,NaN,"DOMAIN 46..315; /note=""Protein kinase""; /evide...",NaN,"Protein kinase superfamily, Ser/Thr protein ki...","REGION 1..25; /note=""Disordered""; /evidence=""E...","ACT_SITE 176; /note=""Proton acceptor""; /eviden...",ACTIVITY REGULATION: Stimulated by clathrin. {...,"BINDING 52..60; /ligand=""ATP""; /ligand_id=""ChE..."
3,P35508,CSNK1D,Bos taurus (Bovine),Casein kinase I isoform delta (CKI-delta) (CKI...,NaN,"DOMAIN 9..277; /note=""Protein kinase""; /eviden...",NaN,"Protein kinase superfamily, CK1 Ser/Thr protei...","REGION 278..364; /note=""Centrosomal localizati...","ACT_SITE 128; /note=""Proton acceptor""; /eviden...",ACTIVITY REGULATION: Drug-mediated inhibition ...,"BINDING 15..23; /ligand=""ATP""; /ligand_id=""ChE..."
4,Q01314,AKT1,Bos taurus (Bovine),RAC-alpha serine/threonine-protein kinase (EC ...,DOMAIN: Binding of the PH domain to phosphatid...,"DOMAIN 5..108; /note=""PH""; /evidence=""ECO:0000...",NaN,"Protein kinase superfamily, AGC Ser/Thr protei...","REGION 450..480; /note=""Disordered""; /evidence...","ACT_SITE 274; /note=""Proton acceptor""; /eviden...",NaN,"BINDING 14..19; /ligand=""1D-myo-inositol 1,3,4..."
...,...,...,...,...,...,...,...,...,...,...,...,...
0,Q29548,HEXB,Sus scrofa (Pig),Beta-hexosaminidase subunit beta (EC 3.2.1.52)...,NaN,NaN,NaN,Glycosyl hydrolase 20 family,NaN,"ACT_SITE 329; /note=""Proton donor""; /evidence=...",ACTIVITY REGULATION: Addition of GM2A stimulat...,NaN
0,P74299,ppc,Synechocystis sp. (strain PCC 6803 / Kazusa),Phosphoenolpyruvate carboxylase (PEPC) (PEPCas...,NaN,NaN,NaN,PEPCase type 1 family,NaN,"ACT_SITE 203; /evidence=""ECO:0000250""; ACT_SIT...",NaN,NaN
0,P39669,glgC,Rhizobium radiobacter (Agrobacterium tumefacie...,Glucose-1-phosphate adenylyltransferase (EC 2....,NaN,NaN,NaN,Bacterial/plant glucose-1-phosphate adenylyltr...,NaN,NaN,NaN,"BINDING 107; /ligand=""alpha-D-glucose 1-phosph..."
1,Q8U8L5,glgC,Agrobacterium fabrum (strain C58 / ATCC 33970)...,Glucose-1-phosphate adenylyltransferase (EC 2....,NaN,NaN,NaN,Bacterial/plant glucose-1-phosphate adenylyltr...,NaN,NaN,NaN,"BINDING 107; /ligand=""alpha-D-glucose 1-phosph..."


In [24]:
df_features.to_csv('protein_features_new.txt')

##### Get InterPro ids for every UniProt entry

In [44]:
#Define function for retrieving the InterPro ids
def retrieve_interpro_ids(uniprot_entry):
    url = "http://www.uniprot.org/uniprot/{}.xml".format(uniprot_entry)
    try:
        handle = urllib.request.urlopen(url)
        record = SeqIO.read(handle, "uniprot-xml")
        interpro_ids = list(filter(lambda ref: 'InterPro' in ref, record.dbxrefs))  
    except:
        interpro_ids =  [0]
    return interpro_ids

In [1]:
all_entries = set(df_features['Entry'])

f = open("./all_entries.txt", mode="x")

for entry in all_entries:
    f.write(entry + "\n")

f.close()

NameError: name 'df_features' is not defined

In [24]:
import multiprocessing as mp


def retrieve_interpro_ids(uniprot_entry):
    url = f"http://www.uniprot.org/uniprot/{uniprot_entry}.xml"
    try:
        handle = urllib.request.urlopen(url)
        record = SeqIO.read(handle, "uniprot-xml")
        interpro_ids = [ref for ref in record.dbxrefs if 'InterPro' in ref]
    except:
        interpro_ids = [0]
    return [uniprot_entry, interpro_ids]

# main
global_list = mp.Manager().list()


with mp.Pool(6) as pool:
    results = [pool.apply_async(retrieve_interpro_ids, args=(entry, global_list, i)) for i, entry in enumerate(all_entries)]
    # wait for all processes to finish
    for r in results:
        r.wait()
return results



In [ ]:
global_list: [[<entry>, [ids..]]]

In [ ]:
#Apply function to dataframe
df_features['Interpro_ids'] = df_features['Entry'].apply(lambda entry: retrieve_interpro_ids(entry))

In [46]:
#Define function for extracting the EC number from the protein name
def extract_EC(string):
    ECs = []
    array = string.split('(')
    for word in array:
        if word.startswith('EC'):
            ECs.append(word.strip(') '))
    return ECs

In [70]:
#Apply function to dataframe to extract EC numbers
df_features['EC numbers'] = df_features['Protein names'].apply(lambda string: extract_EC(string))

C:\Users\Elin\AppData\Local\Temp\ipykernel_11976\3852558869.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_features['EC numbers'] = df_features['Protein names'].apply(lambda string: extract_EC(string))


In [ ]:
#Keep only relevant columns
df_features_small = df_features[['Entry', 'Organism', 'EC numbers', 'Interpro_ids']]

In [ ]:
#Make sure that every element of the EC number and InterPro id lists are evaluated as strings
df_features_small['EC numbers'] = df_features_small['EC numbers'].apply(lambda string: ast.literal_eval(string))
df_features_small['Interpro_ids'] = df_features_small['Interpro_ids'].apply(lambda string: ast.literal_eval(string))

In [ ]:
#Explode on EC numbers to get one Entry, Organism, EC number and InterPro id on each row
df_features_new = df_features_small.explode('EC numbers')

In [ ]:
#Remove 'EC ' from EC numbers
df_features_new['EC numbers'] = df_features_new['EC numbers'].str.replace('EC ', '')

In [ ]:
#Extract organism name from Organism columns
df_features_new['Org'] = df_features_new['Organism'].apply(lambda x: ' '.join(x.split()[:2]))

In [ ]:
#Save to csv
df_features_new.to_csv('protein_domains.txt')

### Merge the dataframe of InterPro ids for each EC numbers with interactions dataframe

In [ ]:
#Merge the dataframes
df_merged = pd.merge(df_brenda_top_hundred, df_features_new, left_on=['EC', 'Org'], right_on=['EC numbers', 'Org'], how='left')

#### Clean up dataframe to get one feature per row

In [ ]:
#Drop unnecessary columns
df_merged_new = df_merged.drop(['Organism', 'Entry', 'EC numbers', 'Enz'], axis=1)

In [ ]:
#Explode on InterPro ids
df_merged_new = df_merged_new.explode('Interpro_ids')

In [ ]:
#Drop duplicates and reset index
df_merged_new.drop_duplicates(inplace=True)
df_merged_new.reset_index(drop=True, inplace=True)

In [ ]:
#Remove 'InterPro' from ids
df_merged_new['Interpro_ids'] = df_merged_new['Interpro_ids'].str.replace('InterPro:', '')

In [ ]:
#Save to csv
df_merged_new.to_csv('features_interactions_merged.txt')

### Map InterPro ids to entry type and entry name

In [103]:
#Get list of entry type and name for InterPro ids, downloaded from the InterPro website
df_entry_list = pd.read_csv('entry.list.txt', sep='\t', header=0)

In [ ]:
#Merge dataframe of features and interactions with the entry-list
df_merged_type = pd.merge(df_merged_new, df_entry_list, left_on='Interpro_ids', right_on='ENTRY_AC', how='left').drop('ENTRY_AC', axis=1)

In [ ]:
#Save file to csv
df_merged_type.to_csv('features_interactions_merged_types.txt')

### Get the features/InterPro ids for organisms and EC numbers in phylogenetic tree

###### This part retrieves the InterPro ids for the Organism-EC number pairs that are mapped in the phylogenetic tree used to identify predicted interactions. The reasoning for doing so is that not all of these EC numbers were documented in BRENDA for all top hundred organisms, and therefore we did not have data on the features for these organisms, resulting in false-negative predictions. 

In [97]:
#Create list of the EC numbers mapped on the phylogenetic tree
top_ten_ECs = ['1.3.5.1', '2.2.1.6', '2.7.1.1', '2.7.1.11', '2.7.1.30', '2.7.1.40', '2.7.2.4', '2.7.7.27', '3.1.3.11', '6.4.1.1']

In [98]:
#Make a dataframe of the top hundred organisms 
df_top_hundred = pd.DataFrame(top_hundred_list, columns=['Org'])

In [101]:
#Associate every organism with every EC number in the top ten group
df_top_hundred['EC'] = [top_ten_ECs]*100
df_top_hundred_EC = df_top_hundred.explode('EC').reset_index(drop=True)

In [ ]:
#Apply string-formula to dataframe to get entry-strings
df_top_hundred_EC['Entry_string'] = df_top_hundred_EC.apply(lambda x: entry_formula(x['EC'], x['Org']), axis=1)

In [ ]:
#Get info for every EC number+Organism combination

df = pd.DataFrame([])

for entry in df_top_hundred_EC['Entry_string']:
    df = pd.concat([df,u.get_df(entry)])

In [ ]:
#Make dataframe with only wanted columns
df_features_topEC = df[columns_of_interest]

In [ ]:
#Get InterPro ids for all features and extract EC numbers
df_features_topEC['Interpro_ids'] = df_features_topEC['Entry'].apply(lambda entry: retrieve_interpro_ids(entry))
df_features_topEC['EC numbers'] = df_features_topEC['Protein names'].apply(lambda string: extract_EC(string))

In [ ]:
#Make dataframe with only wanted columns
df_features_topEC_small = df_features_topEC[['Entry', 'Organism', 'EC numbers', 'Interpro_ids']]

In [ ]:
#Explode dataframe to get one EC number per row
df_features_topEC_exp = df_features_topEC_small.explode('EC numbers')

In [ ]:
#Clean up EC numbers and organism names
df_features_topEC_exp ['EC numbers'] = df_features_topEC_exp ['EC numbers'].str.replace('EC ', '')
df_features_topEC_exp['Org'] = df_features_topEC_exp['Organism'].apply(lambda x: ' '.join(x.split()[:2]))

In [ ]:
#Save file to csv
df_features_topEC_exp.to_csv('features_for_ECs_in_tree.txt')